# Natural_Language_Processing/Assignment_1/MDS201803

Basic imports

In [332]:
import numpy as np

Dictionary Creation

In [57]:
# This dictionary stores the words which are the building blocks of numerical figures
# We are considering numbers in the range 0-9999999

numeric_1D = {'zero' : 0, 'one' : 1, 'two' : 2, 'three' : 3, 'four' : 4, 'five' : 5, 'six' : 6, 'seven' : 7, 'eight' : 8, 'nine' : 9}
numeric_2D_1 = {'ten' : 10, 'eleven' : 11, 'twelve' : 12, 'thirteen' : 13, 'fourteen' : 14, 'fifteen' : 15, 'sixteen' : 16, 'seventeen' : 17, 'eighteen' : 18, 'nineteen' : 19} 
numeric_2D_2 = {'twenty' : 20, 'thirty' : 30, 'fourty' : 40, 'fifty' : 50, 'sixty' : 60, 'seventy' : 70, 'eighty' : 80, 'ninety' : 90}

numeric_dict = {**numeric_1D, **numeric_2D_1, **numeric_2D_2}

In [3]:
# This dictionary contains the denominations required to write the word forms of numerical in the desired range

denomination_dict = {'hundred' : 100, 'thousand' : 1000, 'lakh' : 100000, 'million' : 1000000}

In [4]:
# This is the list containing the conjunctions required to build the words

conjunction_list = ['and', 'point', 'by']

In [564]:
# This dictionary contains the basic physical quantities units

base_units_dict = {'seconds' : 's', 'meters' : 'm', 'kilogram' : 'kg', 'kilos' : 'kg', 'ampere' : 'A', 'kelvin' : 'K', 
                   'mole' : 'mol', 'candela' : 'cd'}

In [569]:
# This dictionary contains the derived physical quantities units

special_units_dict = {'radian' : 'rad', 'steradian' : 'sr', 'hertz' : 'Hz', 'newton' : 'N', 'pascal' : 'Pa',
                      'joule' : 'J', 'watt' : 'W', 'coulomb' : 'C', 'volt' : 'V', 'farad' : 'F', 'ohm' : 'Ohm', 
                      'siemens' : 'S', 'weber' : 'Wb', 'tesla' : 'T', 'henry' : 'H', 'degree Celsius' : '°C',
                      'lumen' : 'lm', 'lux' : 'lx', 'becquerel' : 'Bq', 'gray' : 'Gy', 'sievert' : 'Sv', 'katal' : 'kat'}

all_units_dict = {**base_units_dict , **special_units_dict}

In [540]:
# This function does the actual elementary computation required for our purpose
# It multiplies the denominations appropriately with the numericals 
# It takes care of the combinatio of the 1D and 2D numericals

def Evaluate(l):
    output = 0
    if len(l) == 1:
        return numeric_dict[l[0]]
    elif l[0] in numeric_1D.keys() and l[1] in numeric_1D.keys():
        index = len(l)
        for i in l:
            output = output + numeric_1D[i]*10**(index-1)
            index = index - 1
    elif l[0] in numeric_1D.keys() and l[1] in list(numeric_2D_1.keys()) + list(numeric_2D_2.keys()):
        output = output + numeric_1D[l[0]]*100 
        for i in l[1:]:
            output = output + numeric_dict[i]
    elif l[1] in denomination_dict.keys():
        output = output + denomination_dict[l[1]] * numeric_dict[l[0]]
    elif l[0] in numeric_2D_2.keys() and l[1] in numeric_1D.keys() and len(l) == 2:
        output = output + numeric_dict[l[0]] + numeric_dict[l[1]]
    elif l[-1] in denomination_dict.keys():
        output = output + Evaluate(l[:-1])*denomination_dict[l[-1]]
    return output 

In [488]:
# This function splits the sequence of words by the denomination/Conjunction words
# It Splits the given sequence into elementary postions each of which is transformed into numericals using "Evaluate"

# This function identifes and adds the 'and' conjunction whereever required (for generalization purposes)
# So that for e.g. ['One hundred one'] & ['One hundred and one'] are treated equally

def Split(l):
    output = []
    last_index = 0
    l_copy = l[:]
    last = False
    for i in l:
        if i in conjunction_list or i in denomination_dict.keys():
            index = l.index(i)
            if i in conjunction_list:
                if last :
                    last_index = index+1
                else:    
                    output.append(Evaluate(l[last_index:index]))
                    output.append(i)
                    last_index = index+1
            else:
                last = True
                output.append(Evaluate(l[last_index:index+1]))
                output.append('and')
                last_index = index+1
        else:
            last = False

    output.append(Evaluate(l[last_index:]))
    return output

In [550]:
# Takes into account the conjunctions and combines the words accordingly

def Compute(l):
    output = 0
    for i in l:
        if isinstance(i,int):
            output = output + i
        
        # The conjunction 'and' simply denotes addition of the two portions 
        
        elif i == 'and':
            pass
        
        # The conjunction 'point' denotes addition of the decimal version of the latter portion with the 1st part
        
        elif i == 'point':
            index = l.index(i)
            size = len(str(l[index+1:][0]))
            output = output + l[index+1:][0]/10**size
            break
            
        # The conjunction 'by' denotes division of the first portion by the later one
            
        elif i == 'by' :
            index = l.index(i)
            output = output/l[index+1:][0]
            break
            
    return output

In [553]:
# This function converts the word array to numeric
# Uses the functions Split --> Evaluate --> Compute
# It also transforms the units to the acronyms

def Convert(l):
    if l[-1] in all_units_dict.keys():
        temp = l[:-1]
        output = [Compute(Split(temp)), all_units_dict[l[-1]]]
    else:
        output = Compute(Split(l))
    return output   

In [ ]:
# The following function takes a text input (string) and does the following :
# -> separates the words w.r.t blankspace (" ")
# -> Scans the list of words once and searches for the keywords in the "numeric_dict"
# -> It also takes into account the conjunctions used in creating the numeric words as well
# -> Finally outputs the string with numericals and unit acronyms

def Extract_n_replace(l):
    
    # "list_words" denotes the blankspace separated words in the given array
    # "temp" is a list of list that stores the list of words which construct a perticular numerical figure
    # "item_index" stores the indices of the numerical figure as a whole
    
    lower = l.lower()
    
    output = []
    list_words = list(lower.split(" "))
    main = l.split(" ")
    copy = list_words.copy()
    
    temp = []
    last = False
    item_index = 0
    
    for i in copy:

        if i in numeric_dict.keys() or (i in conjunction_list and last) or i in denomination_dict.keys():
        # This condition checks whether the word is numeric, a numeric conjunction or a numeric denomination
        # Once the condition is satisfied it turns the "last" flag to True
        # The "last" flag is used to denote that a sequence of numeric words has started
        
            last = True
            try:
                temp[item_index].append(i)
            except IndexError:
                temp.append([i]) 
                
        elif i in all_units_dict.keys():
        # This condition takes into account the physical quantities unit (if any) present
        
            temp[item_index].append(i)
            for j in list(Convert(temp[item_index])):
                output.append(j)
            item_index = item_index + 1
            last = False
        
        elif last:            
        # This condition terminates the current numeric word 
            for j in [Convert(temp[item_index])]:
                output.append(j)
            output.append(i)
            item_index = item_index + 1
            last = False
        
        else:
            output.append(main[copy.index(i)])
        
    print(' '.join(str(word) for word in output))

### A few testing examples

In [706]:
Extract_n_replace('A car starts from rest and accelerates uniformly over a time of five point two one seconds for a distance of one hundred and ten kilos . Determine the acceleration of the car.')

A car starts from rest and accelerates uniformly over A time of 5.21 s for A distance of 110 kg . Determine the acceleration of the car.


In [702]:
Extract_n_replace('An airplane accelerates down a runway at three point two zero m/s2 for thirty two point eight s until is finally lifts off the ground. Determine the distance traveled before takeoff.')

An airplane accelerates down a runway at 3.2 m/s2 for 32.8 s until is finally lifts off the ground. Determine the distance traveled before takeoff.


In [703]:
Extract_n_replace('Upton Chuck is riding the Giant Drop at Great America. If Upton free falls for two point six zero seconds, what will be his final velocity and how far will he fall? ')

Upton Chuck is riding the Giant Drop at Great America. If Upton free falls for 2.6 seconds, what will be his final velocity and how far will he fall? 


In [707]:
Extract_n_replace('A race car accelerates uniformly from Eighteen point five m/s to fourty six point one m/s in two point four seconds . Determine the acceleration of the car and the distance traveled. ')

A race car accelerates uniformly from 18.5 m/s to 46.1 m/s in 2.4 s . Determine the acceleration of the car and the distance traveled. 


In [708]:
Extract_n_replace('A bike accelerates uniformly from rest to a speed of seven point one m/s over a distance of thirty five point four m. Determine the acceleration of the bike. ')

A bike accelerates uniformly from rest to A speed of 7.1 m/s over A distance of 35.4 m. Determine the acceleration of the bike. 
